In [188]:
#import os
import math
import fiona 
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Polygon, Point
from explode import explode

In [189]:
#SET COUNTRY <----------------------------------------------------------------------------------CHANGE
# n = 'TUR' 
# n = 'GRC'
# n = 'ESP'
# n = 'HRV' 
# n = 'ITA' 
# n = 'CYP' 
# n = 'MLT' 
# n = 'FRA' 
n = 'ALB' 

#SET CRS #<-------------------------------------------------------------------------------------CHANGE
# crs = 'EPSG:5637' #turkey
# crs = 'EPSG:2100' #greece
# crs = 'EPSG:2062' #spain
# crs = 'EPSG:3765' #croatia
# crs = 'EPSG:7794' #italy
# crs = 'EPSG:6312' #cyprus
# crs = 'EPSG:3034' #malta
# crs = 'EPSG:2154' #france
crs = 'EPSG:6962' #albania
wgs84= {'init' :'EPSG:4326'}

#open file
f = '../targets_2_simrdwn/' + n + '/'
file = f + 'predictions_' + n + '.csv'
detections = pd.read_csv(file, delimiter=',', header=0)
print(len(detections), 'total SIMRDWN predictions')
#detections = detections[0:1000]
detections.head()

2217 total SIMRDWN predictions


,img_file,img_width,img_height,label,xmin,ymin,xmax,ymax
0,image_18_41_44254_19_48414_41_45105_19_47158.jpeg,2341,2116,c,2215,1544,2338,1806
1,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,c,1008,374,1114,521
2,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,c,1039,23,1249,272
3,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,c,94,658,176,783
4,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,c,996,1906,1087,2086


In [190]:
x = detections['img_file'].iloc[1]
x.split('_')

['image',
 '18',
 '41',
 '37464',
 '19',
 '48442',
 '41',
 '38315',
 '19',
 '47188.jpeg']

In [191]:
#georeference prediction bounding boxes and convert to centroid
selected = detections

lon_list, lat_list, radius_list, diameter_list = [], [], [], []
for i in range(len(selected)):
    img = detections['img_file'].iloc[i]

#     #extract bottom right coordinate from image name
    br_lon = float(img.split('_')[4] + '.' + img.split('_')[5]) #lon = x                                       
    br_lat = float(img.split('_')[2] + '.' + img.split('_')[3]) #lat = y
    
    #extract top left coordinate from image name
    tl_lon = float(img.split('_')[8] + '.' + (img.split('_')[9]).split('.')[0]) #lon = x
    tl_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y    
    
#     #FOR GREECE ONLY - extract bottom right coordinate from image name
#     br_lon = float(img.split('_')[8] + '.' + img.split('_')[9]) #lon = x                                       
#     br_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y
    
#     #extract top left coordinate from image name
#     tl_lon = float(img.split('_')[12] + '.' + (img.split('_')[13]).split('.')[0]) #lon = x
#     tl_lat = float(img.split('_')[10] + '.' + img.split('_')[11]) #lat = y
    
    #image dimensions
    img_w = detections['img_width'].iloc[i]
    img_h = detections['img_height'].iloc[i]
    width = abs(br_lon - tl_lon) 
    height = abs(tl_lat - br_lat)
    
    #decimal degrees per pixel
    res_x = width / img_w #image width
    res_y = height / img_h #image height
    
    #res
    lat = tl_lat * math.pi / 180
    res = 156543.04 * math.cos(lat) / (2 ** 18)
    
    #convert bounding box to centroid
    xmin = detections['xmin'].iloc[i]
    ymin = detections['ymin'].iloc[i]
    xmax = detections['xmax'].iloc[i]
    ymax = detections['ymax'].iloc[i]
    x = (xmin + xmax) / 2
    y = (ymin + ymax) / 2
    
    #convert centroid point to lat/lon   
    x_center = tl_lon + (x * res_x) 
    y_center = tl_lat - (y * res_y) 
    
    #estimate radius of detection
    w = xmax-xmin 
    h = ymax-ymin 
    radius = (((w+h)/2)/4)*1.1
    diameter = radius * 2
    lon_list.append(x_center)
    lat_list.append(y_center)
    radius_list.append(radius)
    diameter_list.append(diameter)

In [192]:
#add attributes to df
d2 = selected.reset_index(drop=True) #super important
d2 = d2.drop(columns=['label'])
d2['x']=lon_list
d2['y']=lat_list
d2['radius']=np.round(radius_list,2)
d2['diameter']=np.round(diameter_list,2)
print(len(d2), 'total SIMRDWN predictions')

#add geometry information to df 
geometry = [Point(i) for i in zip(lon_list, lat_list)]
d2 = gpd.GeoDataFrame(d2, geometry=geometry, crs=wgs84)
d2 = d2.to_crs({'init': crs})
print(d2.crs)
d2.head()

2217 total SIMRDWN predictions
{'init': 'EPSG:6962'}


,img_file,img_width,img_height,xmin,ymin,xmax,ymax,x,y,radius,diameter,geometry
0,image_18_41_44254_19_48414_41_45105_19_47158.jpeg,2341,2116,2215,1544,2338,1806,19.483794,41.444314,52.94,105.88,POINT (-43112.932756701 49442.62351610343)
1,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,1008,374,1114,521,19.477573,41.381349,34.79,69.58,POINT (-43674.2944465127 42456.89599060397)
2,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,1039,23,1249,272,19.478019,41.382556,63.11,126.22,POINT (-43636.26146729386 42590.71697841908)
3,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,94,658,176,783,19.472604,41.380250,28.46,56.92,POINT (-44090.41065260505 42337.4135027239)
4,image_18_41_37464_19_48442_41_38315_19_47188.jpeg,2337,2114,996,1906,1087,2086,19.477469,41.375115,37.26,74.53,POINT (-43687.18223877608 41765.06040136438)


In [193]:
#select predictions within search area
# search_area = gpd.read_file('../0_search_areas/4_search_area/search_area_100m_' + n + '.shp')
# search_area = search_area.to_crs({'init': crs})

# d2 = sjoin(d2, search_area, how='inner', op='within')
# print(len(d2), 'predictions within search area')

#select predictions by diameter
d = 55
d2 = d2[d2['diameter'] <= d]
geometry = [Point(i) for i in zip(d2['x'], d2['y'])]

d2 = d2[['radius', 'geometry']]
print(len(d2), 'total SIMRDWN predictions less than or equal to', d, 'meters')
d2.head()

700 total SIMRDWN predictions less than or equal to 55 meters


,radius,geometry
10,17.74,POINT (-51589.88248328368 -19666.54175295102)
11,27.36,POINT (-52354.78348316495 -19486.79100134856)
19,16.91,POINT (-43987.08548226559 35418.37555470552)
22,17.88,POINT (-48495.62414856638 38506.95445024451)
29,17.19,POINT (-40068.36376366818 53537.83757437248)


In [194]:
#aggregate predictions 
buffer = gpd.GeoDataFrame(geometry = d2.buffer(10)) #buffer by 10 meters
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
print(len(d2), 'predictions aggregated to', len(buffer_exploded), 'predictions')

700 predictions aggregated to 296 predictions


In [195]:
#calculate mean radius of aggregated predictions
d2_ag = gpd.sjoin(d2, buffer_exploded, how="inner", op='intersects')
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid, crs=wgs84)
centroids['radius']=d2_ag.groupby('index_right')['radius'].mean()
centroids.crs={'init' : crs}
print(len(centroids), 'predictions')
centroids.head()

296 predictions


,geometry,radius
0,POINT (859.7224793015391 -145845.2563593077),9.210
1,POINT (888.9317757049915 -145835.4027366633),10.040
2,POINT (850.2885827214164 -145812.2706766105),8.800
3,POINT (881.5678651815007 -145803.1040765973),8.520
4,POINT (-946.3264018000033 -145740.0547555584),6.626


In [196]:
#generate farmsite polygons
b = 100
buffer = gpd.GeoDataFrame(geometry = centroids.buffer(b)) #buffer by 100m
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
buffer_exploded.crs={'init' : crs}
buffer_exploded.shape

(181, 1)

In [197]:
#add number of predictions per farmsite to farmsite polygons
count=sjoin(centroids, buffer_exploded, how='inner', op='within')
buffer_exploded['count']=count.groupby('index_right')['index_right'].count()
buffer_exploded['count']=count.groupby('index_right')['index_right'].count()
buffer_exploded["farm_id"] = buffer_exploded.index + 1 #add farm ID

print(len(buffer_exploded), 'total farmsite predictions based on farmsites')
buffer_exploded.head()

181 total farmsite predictions based on farmsites


,geometry,count,farm_id
0,POLYGON ((-79.95851347497032 -145395.406937864...,1,1
1,POLYGON ((-1875.345881729254 -143565.130689695...,1,2
2,POLYGON ((727.9327926177494 -145202.3411080966...,5,3
3,POLYGON ((-191.4924003529065 -136096.561167961...,1,4
4,POLYGON ((-929.5054259593334 -145864.487387605...,8,5


In [198]:
#select farmsites by number of predictions
p = 4 #number of predictions per farmsite
farmsites = buffer_exploded[buffer_exploded['count'] >= p]
print(len(farmsites), 'farm site predictions containing', p, 'or more net pen predictions')
farmsites.head()

12 farm site predictions containing 4 or more net pen predictions


,geometry,count,farm_id
2,POLYGON ((727.9327926177494 -145202.3411080966...,5,3
4,POLYGON ((-929.5054259593334 -145864.487387605...,8,5
5,POLYGON ((906.0398282008606 -145933.8374163994...,9,6
12,POLYGON ((-17568.11562848373 -104422.775058502...,4,13
21,POLYGON ((-48690.17864914841 -75820.3922735130...,6,22


In [199]:
#select predictions by farmsites
select = farmsites
select['predictions'] = 0
select = select.dissolve(by='predictions')
mask = centroids.within(select.loc[0, 'geometry'])
farmsites=farmsites.drop(['predictions'], axis=1)
d2_clip = centroids.loc[mask]

print(len(d2_clip), 'net pen predictions within farm sites')
print(d2_clip.crs)
d2_clip.head()

96 net pen predictions within farm sites
{'init': 'EPSG:6962'}


C:\Users\Zack\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,geometry,radius
0,POINT (859.7224793015391 -145845.2563593077),9.210
1,POINT (888.9317757049915 -145835.4027366633),10.040
2,POINT (850.2885827214164 -145812.2706766105),8.800
3,POINT (881.5678651815007 -145803.1040765973),8.520
4,POINT (-946.3264018000033 -145740.0547555584),6.626


In [200]:
#add id and farm_id to predictions
d2_clip_id=sjoin(farmsites, d2_clip, how='right', op='intersects')
d2_clip_id=d2_clip_id.reset_index(drop=True)
d2_clip_id["id"] = d2_clip_id.index + 1 
d2_clip_id['diameter']=d2_clip_id['radius'] * 2
d2_clip=d2_clip_id[['id', 'farm_id', 'radius', 'diameter', 'geometry']]
print(len(d2_clip_id), 'net pen predictions')
d2_clip.head()

96 net pen predictions


,id,farm_id,radius,diameter,geometry
0,1,3,10.176667,20.353333,POINT (628.0997068046892 -145198.9434917682)
1,2,3,8.209000,16.418000,POINT (551.8207308695282 -145188.9537607147)
2,3,3,9.280000,18.560000,POINT (627.4383795040678 -145168.3292430414)
3,4,3,6.875714,13.751429,POINT (542.7243397460364 -145144.4473028099)
4,5,3,9.077500,18.155000,POINT (651.9807443236982 -145137.4133908063)


In [201]:
# #generate farmsite extents
envelope = gpd.GeoDataFrame(geometry = farmsites.envelope)
farmsites2=farmsites.copy()
farmsites2['geometry'] = envelope['geometry']
print(len(farmsites2), 'farm site predictions')
farmsites2.head()

12 farm site predictions


,geometry,count,farm_id
2,POLYGON ((442.7243397460364 -145298.9434917682...,5,3
4,POLYGON ((-1063.972293536431 -145870.739244614...,8,5
5,POLYGON ((741.8818061676782 -145945.2563593077...,9,6
12,POLYGON ((-17787.22301770811 -104467.667048287...,4,13
21,POLYGON ((-48924.05438628457 -75873.6489176212...,6,22


In [202]:
#exclusion of false positives
# path= f + n + '_exclude.kml'
# gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
# exclude = gpd.read_file(path, driver='KML')
# exclude = exclude.to_crs({'init': crs})

# #create exclusion mask
# exclude['Dissolve'] = 0
# exclude_dis = exclude.dissolve(by='Dissolve')

# #eliminate net pen false positives
# mask = ~d2_clip.within(exclude_dis.loc[0, 'geometry'])
# d2_clip = d2_clip.loc[mask]
# print('final number of predicted net pens:', len(d2_clip))

# #eliminate farmsite false positives
# mask2 = ~farmsites2.within(exclude_dis.loc[0, 'geometry'])
# farmsites2 = farmsites2.loc[mask2]
# print('final number of predicted farm sites:', len(farmsites2))

In [203]:
# #generate net pen buffers
def buffer(row):
     return row.geometry.buffer(row.radius)   
    
d2_clip2 = d2_clip.copy()
d2_clip2.crs

buff = d2_clip2['geometry'] = d2_clip2.apply(buffer, axis=1)
circles = gpd.GeoDataFrame(d2_clip2, geometry = buff, crs={'init': crs})

In [204]:
#generate farmsite centroids
farmsites3 = farmsites2.copy()
farm_pts = gpd.GeoDataFrame(farmsites3, geometry = farmsites3.centroid)

In [205]:
print(len(circles), circles.crs)
print(len(d2), d2.crs)
print(len(d2_clip), d2_clip.crs)
print(len(farmsites2), farmsites2.crs)

96 {'init': 'EPSG:6962'}
700 {'init': 'EPSG:6962'}
96 {'init': 'EPSG:6962'}
12 {'init': 'EPSG:6962'}


In [206]:
# # #export buffered centroids
# circles.to_file(f + n + '_simrdwn_pens.shp')
# circles['geometry'] = circles['geometry'].to_crs(epsg=4326)
# circles.to_file(f + n + '_simrdwn_pens.geojson', driver='GeoJSON')

# # #export all centroids
# d2 = d2.to_crs({'init': crs})
# d2.to_file(f + n + '_simrdwn.shp')
# d2['geometry'] = d2['geometry'].to_crs(epsg=4326)
# d2.to_file(f + n + '_simrdwn.geojson', driver='GeoJSON')

# #export centroids within farmsites
# d2_clip.to_file(f + n + '_simrdwn_pts.shp')
# d2_clip['geometry'] = d2_clip['geometry'].to_crs(epsg=4326)
# d2_clip.to_file(f + n + '_simrdwn_pts.geojson', driver='GeoJSON')

# #export farmsites
# # farmsites.to_file(f + n + '_simrdwn_farmsites.shp')
# # farmsites['geometry'] = farmsites['geometry'].to_crs(epsg=4326)
# # farmsites.to_file(f + n + '_simrdwn_farmsites.geojson', driver='GeoJSON')

# # #export farmsites extents
# farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.shp')
# farmsites2['geometry'] = farmsites2['geometry'].to_crs(epsg=4326)
# farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.geojson', driver='GeoJSON')

# #export farmsites points
farm_pts.to_file(f + n + '_simrdwn_farm_pts.shp')
farm_pts['geometry'] = farm_pts['geometry'].to_crs(epsg=4326)
farm_pts.to_file(f + n + '_simrdwn_farm_pts.geojson', driver='GeoJSON')